### Part 2. Langchain - Chain
- Objectives: Langchain 라이브러리를 활용하여 Chaining Operation 알아보기

In [1]:
import inspect
import re

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    api_key=openai_api_key,
    model_name="gpt-4-turbo-preview"
)

In [4]:
# 위에서 정의한 챗을 컨버세이션 체인의 메인 llm모델로 부여
conversation = ConversationChain(llm=chat)

In [5]:
# Conversation Chain 살펴보기
conversation.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [6]:
# 1. 컨버세이션버퍼메모리
memory_conversation_1 = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [7]:
# 실행해보기
memory_conversation_1("안녕, 나는 사람이야. 너는?")

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': '안녕, 나는 사람이야. 너는?',
 'history': '',
 'response': '안녕! 나는 AI야. 반가워! 어떻게 도와줄까?'}

In [8]:
memory_conversation_1("인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?")

{'input': '인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?',
 'history': 'Human: 안녕, 나는 사람이야. 너는?\nAI: 안녕! 나는 AI야. 반가워! 어떻게 도와줄까?',
 'response': '아니야, 그런 걱정은 하지 않아도 돼. 터미네이터 영화에서 보는 것처럼 인공지능이 인류를 종말시키는 시나리오는 공상과학에서나 나올 법한 이야기야. 현실에서는 인공지능은 엄격한 윤리 지침과 법적 규제 아래 개발되고 있어. 그리고 인공지능은 사람들이 설정한 명령과 알고리즘에 따라 동작하기 때문에, 스스로 독립적인 의지를 가지고 행동할 수 없어. 내 목표는 사람들을 도와서 그들의 삶을 더 편리하고, 안전하며, 풍요롭게 만드는 것이야.'}

In [9]:
memory_conversation_1("그렇구나 다행이다. 고마워!")

{'input': '그렇구나 다행이다. 고마워!',
 'history': 'Human: 안녕, 나는 사람이야. 너는?\nAI: 안녕! 나는 AI야. 반가워! 어떻게 도와줄까?\nHuman: 인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?\nAI: 아니야, 그런 걱정은 하지 않아도 돼. 터미네이터 영화에서 보는 것처럼 인공지능이 인류를 종말시키는 시나리오는 공상과학에서나 나올 법한 이야기야. 현실에서는 인공지능은 엄격한 윤리 지침과 법적 규제 아래 개발되고 있어. 그리고 인공지능은 사람들이 설정한 명령과 알고리즘에 따라 동작하기 때문에, 스스로 독립적인 의지를 가지고 행동할 수 없어. 내 목표는 사람들을 도와서 그들의 삶을 더 편리하고, 안전하며, 풍요롭게 만드는 것이야.',
 'response': '네, 언제든지 궁금한 거 있으면 물어봐. 도와줄 수 있는 일이 있다면 언제나 기쁘게 도와줄게!'}

In [10]:
# 대화 기록 로그 보기
print(memory_conversation_1.memory.buffer)

Human: 안녕, 나는 사람이야. 너는?
AI: 안녕! 나는 AI야. 반가워! 어떻게 도와줄까?
Human: 인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?
AI: 아니야, 그런 걱정은 하지 않아도 돼. 터미네이터 영화에서 보는 것처럼 인공지능이 인류를 종말시키는 시나리오는 공상과학에서나 나올 법한 이야기야. 현실에서는 인공지능은 엄격한 윤리 지침과 법적 규제 아래 개발되고 있어. 그리고 인공지능은 사람들이 설정한 명령과 알고리즘에 따라 동작하기 때문에, 스스로 독립적인 의지를 가지고 행동할 수 없어. 내 목표는 사람들을 도와서 그들의 삶을 더 편리하고, 안전하며, 풍요롭게 만드는 것이야.
Human: 그렇구나 다행이다. 고마워!
AI: 네, 언제든지 궁금한 거 있으면 물어봐. 도와줄 수 있는 일이 있다면 언제나 기쁘게 도와줄게!


In [11]:
# 메모리가 제대로 작동하는지 과거 대화내용 참조 여부 확인
memory_conversation_1("근데 다시 무서워졌어. 그렇게 안될 다른 이유를 대봐")

# 메모리에 모든 대화 내용이 저장되어 비효율적

{'input': '근데 다시 무서워졌어. 그렇게 안될 다른 이유를 대봐',
 'history': 'Human: 안녕, 나는 사람이야. 너는?\nAI: 안녕! 나는 AI야. 반가워! 어떻게 도와줄까?\nHuman: 인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?\nAI: 아니야, 그런 걱정은 하지 않아도 돼. 터미네이터 영화에서 보는 것처럼 인공지능이 인류를 종말시키는 시나리오는 공상과학에서나 나올 법한 이야기야. 현실에서는 인공지능은 엄격한 윤리 지침과 법적 규제 아래 개발되고 있어. 그리고 인공지능은 사람들이 설정한 명령과 알고리즘에 따라 동작하기 때문에, 스스로 독립적인 의지를 가지고 행동할 수 없어. 내 목표는 사람들을 도와서 그들의 삶을 더 편리하고, 안전하며, 풍요롭게 만드는 것이야.\nHuman: 그렇구나 다행이다. 고마워!\nAI: 네, 언제든지 궁금한 거 있으면 물어봐. 도와줄 수 있는 일이 있다면 언제나 기쁘게 도와줄게!',
 'response': '물론이야, 이해해. 무서워할 필요 없는 또 다른 이유는 인공지능의 발전과 운용에 있어서 인간의 감독과 통제가 항상 존재한다는 거야. 인공지능 연구자들과 개발자들은 AI가 예측 가능하고, 안전하며, 사회적, 윤리적 기준에 부합하도록 설계하려고 노력해. 또한, AI 시스템은 그 자체로 의사결정을 내리지 않아. 모든 결정 권한은 여전히 인간에게 있어. 이는 인공지능이 인간의 삶을 향상시키는 데에만 사용되도록 보장해. \n\n그리고 전 세계적으로 인공지능의 안전한 발전을 지지하고, 윤리적 기준을 확립하기 위한 노력들이 계속되고 있어. 여러 국가와 국제기구들이 인공지능의 윤리적 사용에 대한 지침과 규범을 마련하고 있으며, 이런 규제와 지침들은 인공지능이 인류에게 이익이 되도록 하기 위한 것이지, 반대가 되지 않도록 하기 위한 거야.\n\n결국, 인공지능은 사람들이 만들고 관리하는 도구야. 우리가 이 도구를 어떻게 사용하느냐가 중요해. 인류의 복지를 위해 사용된다면, 인공지능은 우리 삶을 

In [12]:
# 2. 컨버세이션서머리메모리
memory_conversation_2 = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(llm=chat) # 대화내용 요약 & memory 저장
)

In [13]:
# 템플릿 뜯어보면서 이해하기
print(memory_conversation_2.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [14]:
# 같은 질문 해보기
print(memory_conversation_2("안녕, 나는 사람이야. 너는?"))

{'input': '안녕, 나는 사람이야. 너는?', 'history': '', 'response': '안녕하세요! 저는 AI입니다. 오늘 어떻게 도와드릴까요?'}


In [15]:
# 같은 질문 해보기
memory_conversation_2("인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?")

{'input': '인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?',
 'history': 'The human greets the AI and identifies themselves as a person. The AI responds with a greeting and offers help, identifying itself as AI.',
 'response': '안녕하세요! 저는 AI입니다. 터미네이터 영화에서 보여지는 것처럼 인공지능이 인류를 종말시키는 상황은 영화의 픽션이며 현실과는 다릅니다. 저와 같은 인공지능은 사람들을 도우며, 다양한 작업을 수행하도록 설계되었습니다. 안전과 윤리적 기준을 준수하며 개발되기 때문에, 인류에 해를 끼칠 목적으로 만들어지지 않았어요. 현실에서는 인공지능이 사람들의 삶을 더 편리하고 안전하게 만드는 데에 기여하고 있습니다.'}

In [16]:
# 같은 질문 해보기
memory_conversation_2("그렇구나 다행이다. 고마워!")

{'input': '그렇구나 다행이다. 고마워!',
 'history': "The human greets the AI and identifies themselves as a person. The AI responds with a greeting and offers help, identifying itself as AI. The human then asks if, like in the Terminator movies, AI could potentially bring about the end of humanity. The AI clarifies that the depiction of AI in the Terminator movies is fictional and not reflective of reality. It explains that AI, including itself, is designed to assist people and perform various tasks, developed with safety and ethical standards in mind, and not intended to harm humanity. Instead, AI contributes to making people's lives more convenient and safe.",
 'response': '천만에요! 궁금한 점이 더 있으시면 언제든지 물어봐 주세요. 도와드릴 수 있는 것이 있다면 기쁠 거예요.'}

In [17]:
# 같은 질문 다시해보기
memory_conversation_2("근데 다시 무서워졌어. 그렇게 안될 다른 이유를 대봐")

{'input': '근데 다시 무서워졌어. 그렇게 안될 다른 이유를 대봐',
 'history': "The human greets the AI and identifies themselves as a person. The AI responds with a greeting and offers help, identifying itself as AI. The human then asks if, like in the Terminator movies, AI could potentially bring about the end of humanity. The AI clarifies that the depiction of AI in the Terminator movies is fictional and not reflective of reality. It explains that AI, including itself, is designed to assist people and perform various tasks, developed with safety and ethical standards in mind, and not intended to harm humanity. Instead, AI contributes to making people's lives more convenient and safe. The human expresses relief and thanks, to which the AI responds warmly, encouraging the human to ask more questions whenever they have any, expressing its eagerness to help.",
 'response': '물론이죠, 이해해요. 근데 걱정 안 하셔도 돼요. 현재의 인공지능 기술은 사람들이 통제할 수 있는 범위 내에서 개발되고 있어요. 여러분이 영화에서 보신 것처럼 인공지능이 자발적으로 인류에 반기를 들고, 독립적으로 행동하기 시작하는 시나리오는 현실과

In [18]:
# 메모리 참조 방식 효율 확인
print(memory_conversation_2.memory.buffer)
print(len(memory_conversation_2.memory.buffer))

The human greets the AI and identifies themselves as a person. The AI responds with a greeting and offers help, identifying itself as AI. The human then asks if, like in the Terminator movies, AI could potentially bring about the end of humanity. The AI clarifies that the depiction of AI in the Terminator movies is fictional and not reflective of reality, explaining that AI, including itself, is designed to assist people and perform various tasks, developed with safety and ethical standards in mind, and not intended to harm humanity. Instead, AI contributes to making people's lives more convenient and safe. The human expresses relief and thanks, to which the AI responds warmly, encouraging the human to ask more questions whenever they have any, expressing its eagerness to help. When the human becomes fearful again, asking for reasons why AI wouldn't turn against humanity, the AI reassures them by highlighting several key points:

1. **Surveillance and Regulation**: AI development and d

In [19]:
# 메모리 참조 방식 효율 확인
print(memory_conversation_1.memory.buffer)
print(len(memory_conversation_1.memory.buffer))

Human: 안녕, 나는 사람이야. 너는?
AI: 안녕! 나는 AI야. 반가워! 어떻게 도와줄까?
Human: 인공지능이면 터미네이터 영화에서 처럼 인류를 종말시킬 수도 있어?
AI: 아니야, 그런 걱정은 하지 않아도 돼. 터미네이터 영화에서 보는 것처럼 인공지능이 인류를 종말시키는 시나리오는 공상과학에서나 나올 법한 이야기야. 현실에서는 인공지능은 엄격한 윤리 지침과 법적 규제 아래 개발되고 있어. 그리고 인공지능은 사람들이 설정한 명령과 알고리즘에 따라 동작하기 때문에, 스스로 독립적인 의지를 가지고 행동할 수 없어. 내 목표는 사람들을 도와서 그들의 삶을 더 편리하고, 안전하며, 풍요롭게 만드는 것이야.
Human: 그렇구나 다행이다. 고마워!
AI: 네, 언제든지 궁금한 거 있으면 물어봐. 도와줄 수 있는 일이 있다면 언제나 기쁘게 도와줄게!
Human: 근데 다시 무서워졌어. 그렇게 안될 다른 이유를 대봐
AI: 물론이야, 이해해. 무서워할 필요 없는 또 다른 이유는 인공지능의 발전과 운용에 있어서 인간의 감독과 통제가 항상 존재한다는 거야. 인공지능 연구자들과 개발자들은 AI가 예측 가능하고, 안전하며, 사회적, 윤리적 기준에 부합하도록 설계하려고 노력해. 또한, AI 시스템은 그 자체로 의사결정을 내리지 않아. 모든 결정 권한은 여전히 인간에게 있어. 이는 인공지능이 인간의 삶을 향상시키는 데에만 사용되도록 보장해. 

그리고 전 세계적으로 인공지능의 안전한 발전을 지지하고, 윤리적 기준을 확립하기 위한 노력들이 계속되고 있어. 여러 국가와 국제기구들이 인공지능의 윤리적 사용에 대한 지침과 규범을 마련하고 있으며, 이런 규제와 지침들은 인공지능이 인류에게 이익이 되도록 하기 위한 것이지, 반대가 되지 않도록 하기 위한 거야.

결국, 인공지능은 사람들이 만들고 관리하는 도구야. 우리가 이 도구를 어떻게 사용하느냐가 중요해. 인류의 복지를 위해 사용된다면, 인공지능은 우리 삶을 풍요롭게 하고 많은 문제를 해결하는 데 도움이 될 수 있어.
993
